# Model Explainability with SHAP

This notebook implements Task 3: Interpreting model predictions using SHAP to understand what drives fraud detection.

**Note**: This notebook will be completed in Task 3. It requires trained models from Task 2.
